<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v1/blob/main/200_50_SMA_Trend_Positive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np

def calculate_stock_metrics(tickers):
    # Create an empty DataFrame to store the results
    results = pd.DataFrame(columns=['Symbol', 'Close', 'Max', 'SMA_50', 'SMA_200', 'Slope_50', 'Slope_200', 'ROC_Slope_50', 'ROC_Slope_200', 'Trend_Positive', 'ATH%'])

    for ticker in tickers:
        #print(ticker)
        # Fetch historical data for the past 300 days to have enough data for 200-day SMA
        stock_data = yf.download(ticker+'.NS', period='5y', progress=False)

        stock_data['Close'] = round(stock_data['Close'],2)
        stock_data['Max'] = round(max(stock_data['Close']),2)
        stock_data['ATH%'] = round((stock_data['Max'] - stock_data['Close'])*100/stock_data['Close'],2)

        # Calculate the 50-day and 200-day SMAs
        stock_data['SMA_50'] = round(stock_data['Close'].rolling(window=50).mean(),2)
        stock_data['SMA_200'] = round(stock_data['Close'].rolling(window=200).mean(),2)

        # Calculate the Slope of the SMAs
        stock_data['Slope_50'] = round(stock_data['SMA_50'].diff(),2)
        stock_data['Slope_200'] = round(stock_data['SMA_200'].diff(),2)

        # Calculate the Rate of Change of the Slope (second derivative)
        stock_data['ROC_Slope_50'] = round(stock_data['Slope_50'].diff(),2)
        stock_data['ROC_Slope_200'] = round(stock_data['Slope_200'].diff(),2)

        # Determine if the Rate of Change of the Slope is Positive
        stock_data['Trend_Positive'] = ((stock_data['Slope_50'] > 0) & (stock_data['Slope_200'] > 0) & (stock_data['ROC_Slope_50'] > 0) & (stock_data['ROC_Slope_200'] > 0))

        # Get the latest row (most recent data)
        latest_data = stock_data.iloc[-1]

        # Append the results to the DataFrame
        # Append the results to the DataFrame using concat
        results = pd.concat([results, pd.DataFrame([{
            'Symbol': ticker,
            'Close': latest_data['Close'],
            'Max': latest_data['Max'],
            'SMA_50': latest_data['SMA_50'],
            'SMA_200': latest_data['SMA_200'],
            'Slope_50': latest_data['Slope_50'],
            'Slope_200': latest_data['Slope_200'],
            'ROC_Slope_50': latest_data['ROC_Slope_50'],
            'ROC_Slope_200': latest_data['ROC_Slope_200'],
            'Trend_Positive': latest_data['Trend_Positive'],
            'ATH%': latest_data['ATH%']
        }])], ignore_index=True)

    return results

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'X40') | (mypf['Criteria'] == 'X40N')]

stock_list = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ','ADANITRANS', 'EQUITAS', 'MCDOWELL-N', 'ARE&M', 'UJJIVAN', 'ASIANPAINT','DREAMFOLKS', 'PVRINOX']

stock_list = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_metrics = calculate_stock_metrics(stock_list)
cols = ['Symbol', 'Target', 'InFolio', 'Criteria','Strategy']
stock_metrics = pd.merge(stock_metrics, mypf[cols], on= 'Symbol')
stock_metrics['NTT'] = np.where(stock_metrics['Strategy']== 'NTT', stock_metrics["Target"], stock_metrics['Max'])
stock_metrics['LTT'] = np.where(stock_metrics['Strategy']== 'BTT', stock_metrics["Target"], stock_metrics['Max'])

stock_metrics['NTT%'] = round((stock_metrics['NTT'] - stock_metrics['Close'])*100/stock_metrics['Close'],2)
stock_metrics['LTT%'] = round((stock_metrics['LTT'] - stock_metrics['Close'])*100/stock_metrics['Close'],2)
stock_metrics['FTT'] = stock_metrics['LTT']
stock_metrics.loc[stock_metrics['Strategy'] == 'NTT', 'FTT'] = stock_metrics['NTT']
stock_metrics['FTT%'] = stock_metrics['LTT%']
stock_metrics.loc[stock_metrics['Strategy'] == 'NTT', 'FTT%'] = stock_metrics['NTT%']

stock_metrics = stock_metrics.drop(columns=['NTT', 'LTT', 'NTT%', 'LTT%', 'ATH%', 'Target', 'Max'])

stock_metrics = stock_metrics[stock_metrics['Trend_Positive'] == True]

stock_metrics.sort_values(by = 'FTT%', ascending=False).head(30)


<ipython-input-3-8d9566f21bb5>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([{


,Symbol,Close,SMA_50,SMA_200,Slope_50,Slope_200,ROC_Slope_50,ROC_Slope_200,Trend_Positive,InFolio,Criteria,Strategy,FTT,FTT%
123,JCHAC,2171.95,1970.38,1417.82,6.75,5.45,3.61,1.01,True,1.0,X40N,BTT,3542.00,63.08
164,PEL,1064.20,973.44,909.50,3.54,0.31,0.54,0.04,True,NaN,NaN,NaN,1697.95,59.55
186,SBICARD,766.05,722.19,728.42,0.81,0.06,0.57,0.11,True,1.0,XSP,NTT,1075.00,40.33
50,CDSL,1421.45,1252.23,1029.29,8.16,3.16,0.23,0.09,True,0.0,XR,BTT,1985.00,39.65
89,GREENPANEL,398.15,345.72,345.77,1.72,0.32,0.29,0.06,True,1.0,XSP,NTT,537.00,34.87
78,FINEORG,5403.15,5290.71,4642.14,13.21,5.79,4.15,0.82,True,1.0,X40N,ATH,7196.00,33.18
192,SHARDACROP,569.30,531.78,428.72,2.46,0.81,0.39,0.08,True,NaN,XR,ATH,748.10,31.41
109,IGL,555.45,531.51,456.88,1.69,0.77,0.28,0.03,True,NaN,AR,BTT,707.00,27.28
197,SONATSOFTW,676.10,645.81,683.73,1.42,0.23,0.17,0.03,True,NaN,MGC,ATH,857.65,26.85
102,HINDUNILVR,2794.30,2675.77,2486.98,7.06,1.49,0.92,0.03,True,1.0,X40,BTT,3513.00,25.72


In [4]:
# rectify
stock_metrics[stock_metrics['FTT%'] < 0].sort_values(by = 'FTT%', ascending=True)

,Symbol,Close,SMA_50,SMA_200,Slope_50,Slope_200,ROC_Slope_50,ROC_Slope_200,Trend_Positive,InFolio,Criteria,Strategy,FTT,FTT%
58,COLPAL,3635.40,3252.96,2742.15,16.18,7.59,0.70,0.02,True,NaN,X40,NTT,2100.0,-42.23
49,CASTROLIND,268.50,251.49,201.70,1.16,0.67,0.12,0.02,True,0.0,XR,NTT,172.0,-35.94
205,SYMPHONY,1539.50,1341.28,1041.56,7.50,3.37,1.57,0.22,True,0.0,X40N,NTT,1201.0,-21.99
95,HDFCAMC,4459.85,4181.68,3693.73,10.94,8.42,0.72,0.25,True,0.0,X40,BTT,3600.0,-19.28
59,CROMPTON,469.60,437.86,345.26,0.86,0.93,0.14,0.02,True,0.0,AR,NTT,382.0,-18.65
106,ICICIGI,2208.10,1950.58,1668.93,9.02,4.15,0.27,0.10,True,0.0,X40,BTT,1893.0,-14.27
21,AVANTIFEED,679.20,657.69,534.56,1.27,1.42,0.52,0.10,True,0.0,XR,NTT,615.0,-9.45
135,LICI,1062.30,1075.85,950.79,0.77,2.25,0.15,0.01,True,0.0,X200,BTT,967.0,-8.97
8,AKZOINDIA,3509.40,3068.37,2693.72,11.82,5.34,0.73,0.38,True,NaN,X40,NTT,3300.0,-5.97
127,JSWHL,7582.50,7023.69,6313.10,17.22,15.15,9.97,2.09,True,NaN,XSP,BTT,7487.0,-1.26
